In [7]:
# Data management
import pandas as pd

# Data preprocessing and trasformation (ETL)
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler, FunctionTransformer, Binarizer, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Math and Stat modules
import numpy as np

# Supervised Learning
from sklearn.linear_model import Perceptron

# Unsupervised Learning

# Visualization
import matplotlib.pyplot as plt

%matplotlib inline

Prepariamo velocemente i dati

In [8]:
credit_card_data = pd.read_csv('BankChurnersMissingData.csv')
credit_card_data.dropna(subset=['Total_Revolving_Bal','Months_Inactive_12_mon'],
                   inplace=True
                  )

# Estraggo la  colonna delle label e la rimuovo dal dataset
credit_card_label = credit_card_data['Attrition_Flag'].map(
    {'Existing Customer':0,
     'Attrited Customer':1
    }
).values
credit_card_data.drop(columns=['Attrition_Flag',
                               'CLIENTNUM',
                               'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2',
                               'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1'],
                      inplace=True)

In [3]:
def unknown_imputer(X, missing_value = 'Unknown'):
    X = X.values
    unique_values, count = np.unique(X,return_counts=True)
    num_nan = count[unique_values == missing_value]
    counting = count[unique_values != missing_value]
    values = unique_values[unique_values != missing_value]
    X_new = X.copy()
    freq = counting / np.sum(counting)
    X_new[X_new == missing_value] = np.random.choice(values,size=num_nan,p=freq)
    return X_new

ui = FunctionTransformer(unknown_imputer)

customer_age_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

category_pipeline = Pipeline([
    ('imputer', FunctionTransformer(unknown_imputer)),
    ('ordinal', OneHotEncoder())
])

features_robust = ['Credit_Limit','Total_Revolving_Bal','Avg_Open_To_Buy']
features_standard = list(set(credit_card_data.select_dtypes(include=['int64','float64']).columns).difference(set(features_robust + ['Avg_Utilization_Ratio', 'Customer_Age'])))

In [4]:
data_preprocessing = ColumnTransformer([
    ('age', customer_age_pipeline, ['Customer_Age']),
    ('gender', OrdinalEncoder(categories=[['M','F']]), ['Gender']),
    ('edu', category_pipeline, ['Education_Level']),
    ('status', category_pipeline, ['Marital_Status']),
    ('income', category_pipeline, ['Income_Category']),
    ('card', category_pipeline, ['Card_Category']),
    ('numeric_robust', RobustScaler(), features_robust),
    ('feature_standard', StandardScaler(), features_standard)
],
    remainder = 'passthrough'
)

In [5]:
feature_matrix = data_preprocessing.fit_transform(credit_card_data)

In [6]:
columns_name = ['Customer_Age','Gender']
for c in ['Education_Level','Marital_Status','Income_Category']:
    cat_inc_name = [c+'_cat{}'.format(i) for i in range(1,len(credit_card_data[c].unique()))]
    columns_name.extend(cat_inc_name)
columns_name.extend(['Card_Category_cat{}'.format(i) for i in range(1,len(credit_card_data['Card_Category'].unique())+1)])
columns_name.extend(features_robust)
columns_name.extend(features_standard)
columns_name.append('Avg_Utilization_Ratio')

# Classificazione

Ora siamo pronti per addestrare un modello ML e lo potremmo fare piuttosto facilmente grazie al fatto che abbiamo gia' preparato i dati e alla struttura di SKL.

Iniziamo ad addestrare un Perceptron.

In [9]:
perceptron = Perceptron()
perceptron.fit(feature_matrix, credit_card_label)

Perceptron()

Vediamo come si comporta con alcuni elementi del training set.

In [13]:
sample_training = feature_matrix[:10,:]
sample_labels = credit_card_label[:10]
prediction_sample = perceptron.predict(sample_training)
for i in range(len(sample_labels)):
    print(i,'True label: {} vs Predicted Label: {}'.format(sample_labels[i],prediction_sample[i]))

0 True label: 0 vs Predicted Label: 0
1 True label: 0 vs Predicted Label: 0
2 True label: 0 vs Predicted Label: 0
3 True label: 0 vs Predicted Label: 0
4 True label: 0 vs Predicted Label: 0
5 True label: 0 vs Predicted Label: 0
6 True label: 0 vs Predicted Label: 0
7 True label: 0 vs Predicted Label: 0
8 True label: 0 vs Predicted Label: 0
9 True label: 0 vs Predicted Label: 0


Un risultato che potrebbe sembrare incoraggiante...

Proviamo a contare quante volte sbaglio su un set di dati gia' visto...

E' fondamentale, infatti, determinare/stimare le performance del modello su un insieme di dati mai visto prima. In precedenza non abbiamo 

In [14]:
all_predicted = perceptron.predict(feature_matrix)

In [21]:
np.sum(all_predicted == credit_card_label)/len(all_predicted)

0.8594567901234568

Non sembra male, ma ...

In [20]:
all_predicted_dummy = np.zeros(len(credit_card_label))
np.sum(all_predicted_dummy == credit_card_label)/len(all_predicted_dummy)

0.8393086419753086

Il perceptron performance un po' meglio di un modello "in tilt".

Questa piccola differenza di performance ci permette di introdurre alcune trappole in cui siamo caduti. 
1) La valutazione delle performance deve essere effettuata su dati non ancora 'visti' dall'algoritmo di apprendimento (errore metodologico)
2) La valutazione deve tenere conto del bilanciamento o meno delle classi

In questa fase, esploriamo il primo problema.

## Cross-validation
Le metodologie di cross-validation possono essere utili per ottenere delle valutazioni piu' attendibili circa le proprieta' di generalizzazione del modello appreso, cioe' quanto il modello sara' in grado di predirre correttamente la classe di appartenenza su dati non ancora osservati.

Per prevenire 

## Performance Measure for Classification